In [1]:
import numpy as np
from numpy.linalg import inv

# Аналитическая модель решения задачи Data Reconciliation

def AnalyticalModel(x, A_y, b_y, c_y, r2_y):
    # Гауссова модель распределения погрешностей.

    # x - вектор уточняемых результатов измерений,
    # A_y - система линейных ограничений на уравнения связи для результатов уточнения, by - вектор правых частей этих уравнений: A_y*y=b_y;
    #     т.к. y=x+dx, то A_y*(x+dx)=b_y. Следовательно, A_y*dx=b_y-A_y*x. Значит, в модели Ефремова-Козлова А=A_y, b=b_y-A_y*x.
    # с_y и r2_y - совокупное ограничение неравенством (y-c_y)'*(y-c_y)<r2_y. Т.к. y=x+dx, то (x+dx-c_y)'*(x+dx-c_y)<r2_y.
    # Следовательно, (dx-(c_y-x))'*(dx-(c_y-x))<r2_y. Значит, в модели Ефремова-Козлова c=c_y-x, r2=r2_y.
    # y - результат уточнения

    x = x.reshape(-1, 1)  # принудительное приведение к вектор-столбцу
    b_y = b_y.reshape(-1, 1)
    c_y = c_y.reshape(-1, 1)

    A = A_y
    b = A_y @ x - b_y
    c = x - c_y
    r2 = r2_y

    # Проверки:
    n = len(x)
    if len(c) != n:
        raise ValueError('Размерность векторов х и с должны совпадать.')
    if A.shape[1] != n:
        raise ValueError('Число столбцов в матрице А должно совпадать с числом элементов в х.')
    if A.shape[0] != len(b):
        raise ValueError('Число строк в матрице А должно совпадать с числом элементов в b.')

    try:
        P = A.T @ inv(A @ A.T)
    except np.linalg.LinAlgError:
        raise ValueError('Матрица A*A^T вырождена, невозможно вычислить обратную матрицу')

    D = np.eye(n) - P @ A

    a = (b.T @ P.T @ P @ b
         - b.T @ P.T @ c
         - c.T @ P @ b
         + c.T @ c
         - c.T @ D @ c
         - r2).item()
    h = 2 * a
    d = (a + c.T @ D @ c).item()

    # Решение квадратного уравнения
    roots = np.roots([a, h, d])

    dx = np.full((n, 2), complex(np.nan))
    for i in range(2):
        lbd = roots[i]
        if np.isclose(lbd, 0) or np.isclose(lbd, -1):
            dx[:, i:i+1] = P @ b
        else:
            dx[:, i:i+1] = P @ b + lbd * (D @ c) / (1 + lbd)

    # Выбор решения с минимальной нормой
    idx = np.argmin(np.sum(dx**2, axis=0))
    y = x - dx[:, idx:idx+1]

    return y

def GetUncertainty(x, dx, A_y, b_y, c_y, r2_y):
    # Все обозначения - те же, что и для AnalyticalModel,
    # dx - массив-вектор cell неопределенностей, формализованных по одному из типов.
    n = len(dx)
    # Оценка частных производных
    alpha = 1e-100
    dy_dx = np.zeros((n, n))

    for i in range(n):
        d = np.zeros((n, 1), dtype=complex)
        d[i] = 1j * alpha
        perturbed = AnalyticalModel(x + d, A_y, b_y, c_y, r2_y)
        dy_dx[:, i] = np.imag(perturbed).flatten() / alpha


    print(dy_dx)

    # Цикл по операциям сложения и умножения по типу неопределенности
    dy = np.zeros(n)
    for i in range(n):
        dy[i] = np.abs(dy_dx[i, 0]) * dx[0] + np.abs(dy_dx[i, 1]) * dx[1]
        for j in range(2, n):
            dy[i] += np.abs(dy_dx[i, j]) * dx[j]

    return dy

######################################################################################
# Пример использования:
# x = [x1;x2]; результат измерения x = [1.0; 0.9; 0.9];
# линейная связь вида 2*x1+3*x2=5; 1*x1+1*x2+1*x3=3
# групповое ограничение: недопустимо R2 при отклонении от столбца [1.0;1.0;1.0], большее, чем 0.2.

x = np.array([[1.0], [0.9], [0.9]])
A = np.array([[2, 3, 0],
              [1, 1, 1]])
b = np.array([[5], [3]])
c = np.array([[1.0], [1.0], [1.0]])
r2 = 0.2

# Уточненный результат Data Reconciliation
y = AnalyticalModel(x, A, b, c, r2)

dx = np.array([[0.1], [0.1], [0.1]])  # пределы абсолютной погрешности (FuzzyInterval)
dy = GetUncertainty(x, dx, A, b, c, r2)

print("Уточненные значения:", y.flatten())
print("Погрешности решений:", dy.flatten())

[[-6.59864878e-15  5.20278077e-15  3.10897875e-15]
 [ 7.16968569e-15 -4.44139822e-15 -1.55448938e-15]
 [ 1.87173211e-15 -1.87173211e-15 -1.01517673e-15]]
Уточненные значения: [1.35856858+0.j 0.76095428+0.j 0.88047714+0.j]
Погрешности решений: [1.49104083e-15 1.31655733e-15 4.75864094e-16]


<ipython-input-1-1bfecd779f71>:87: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dy[i] = np.abs(dy_dx[i, 0]) * dx[0] + np.abs(dy_dx[i, 1]) * dx[1]
<ipython-input-1-1bfecd779f71>:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dy[i] += np.abs(dy_dx[i, j]) * dx[j]
